In [ ]:
# First Load all the required library and also Start Spark Session
# Load all the required library
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import json

URL = 'https://api.data.gov.sg/v1/environment/pm25'

In [ ]:
spark = SparkSession.builder.appName("wasup").getOrCreate()
sqlContext = SparkSession(spark)
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
def get_body_reponse(response):
    if response['api_info']['status'] == 'healthy':
        return response
    
def parameters():
    date_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    return {'date_time':date_time}

def request(url,parameters):
    r = requests.get(url,params=parameters)
    if r.status_code == 200:
        print(f'successful API {r.status_code}')
        return r
    else:
        print(f'error API {r.status_code}')

In [ ]:
r = request(URL,parameters())
content = r.json()
data = get_body_reponse(content)
json_object = json.dumps(data)
print(json_object)

with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
# creating a dataframe 
dataframe = spark.read.format("json").option("header","true").option("inferSchema","true").load("sample.json")
dataframe = dataframe.select(col("items.readings.pm25_one_hourly.central").alias("pm25_central"),
                            col("items.readings.pm25_one_hourly.east").alias("pm25_east"),
                            col("items.readings.pm25_one_hourly.north").alias("pm25_north"),
                            col("items.readings.pm25_one_hourly.south").alias("pm25_south"),
                            col("items.readings.pm25_one_hourly.west").alias("pm25_west"),
                            col("items.timestamp").alias("timestamp"),
                            col("items.update_timestamp").alias("update_timestamp"))
# show data frame 
dataframe.show() 

In [ ]:
#Checking dataframe schema
dataframe.printSchema()